In [6]:
from pymycobot.mycobot import MyCobot
from pymycobot import PI_PORT, PI_BAUD
import time
import csv
import math
import numpy as np
import pandas as pd
from sympy.matrices import Matrix
from scipy.optimize import least_squares
from sympy  import symbols, cos, sin, atan2, pi, lambdify 
import os
from pathlib import Path

In [15]:
#Global variables
speed = 30 # Speed for moving servos
sleepTime = 5 # Sleep time after moving servos
origin = [0, 0, 0, 0, 0, 0] # Origin position for servos
q1, q2, q3, q4, q5, q6 = symbols('q1 q2 q3 q4 q5 q6') # Define symbolic variables for joint angles
right = pi/2

In [17]:
candidate = Path('Lab3/mecharm_control_group_5.csv')
if candidate.exists():
    csv_path = candidate
else:
    # search up from current working dir for the file
    csv_path = None
    for p in Path.cwd().rglob('mecharm_control_group_5.csv'):
        if 'Lab3' in str(p.parent):
            csv_path = p
            break
    if csv_path is None:
        # try absolute known repo path as last resort
        abs_candidate = Path('/Users/yusufdineh/Documents/GitHub/EE347/Lab3/mecharm_control_group_5.csv')
        if abs_candidate.exists():
            csv_path = abs_candidate

if csv_path is None:
    raise FileNotFoundError("mecharm_control_group_5.csv not found. Put it in Lab3/ under the repo or set notebook cwd to the repo root (use %cd).")
print("Loading CSV from:", csv_path)
data = pd.read_csv(csv_path)


Loading CSV from: /Users/yusufdineh/Documents/GitHub/EE347/Lab3/mecharm_control_group_5.csv


In [18]:
#Joint Angles from Lab 2 task 3
known_joint_angles = data[['J1','J2','J3','J4','J5','J6']].to_numpy()
#End-Effector Positions from Lab 2 task 3
known_position = data[['X','Y','Z']].to_numpy()
#End-Effector Orientations from Lab 2 task 3
known_orientation = data[['RX','RY','RZ']].to_numpy()

In [16]:
# Generate Empty DH table for MechArm
# DH table rows: [a, alpha, d, offset, offset]
# a      : link length (mm)
# alpha  : link twist (radians)
# d      : link offset (mm)
# offset : constant offset added to the joint variable (radians)
# q      : symbolic joint variable (q1..q6)
dh_table = [
    [0,         -right,       130,  q1],           # joint 1
    [105,           0,         0,   q2 - right],   # joint 2
    [10,        -right,       0,   q3],           # joint 3
    [0,         right,      100,   q4],           # joint 4
    [0,         -right,       0,   q5],           # joint 5
    [0,         0,         80,   q6]              # joint 6
]

In [ ]:
def get_transformation_matrix(a, alpha, d, theta):
    return Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d],
        [sin(theta)*sin(alpha), cos(alpha)*sin(alpha), cos(alpha), cos(alpha)*d],
        [0, 0, 0, 1]
    ])

In [ ]:
def overall_transformation(dh_table):
    T = Matrix(np.identity(4))
    for i in range(len(dh_table)):
        a, alpha, d, theta = dh_table[i]
        T_i = get_transformation_matrix(a, alpha, d, theta)
        T = T * T_i
    return T

In [ ]:
# Calculate overall transformation matrix
T = overall_transformation(dh_table)
calculated_position = np.zeros((len(known_joint_angles), 3))
for i, q_vals in enumerate(known_joint_angles):
    # Convert joint angles to radians
    q1_val, q2_val, q3_val, q4_val, q5_val, q6_val = np.radians(q_vals)
    
    # Substitute joint angles into transformation matrix
    T_substituted = T.subs({q1: q1_val, q2: q2_val, q3: q3_val, q4: q4_val, q5: q5_val, q6: q6_val})
    end_effector_position = T_substituted[:3, 3]
    calculated_position[i, :] = np.array([float(end_effector_position[0]), float(end_effector_position[1]), float(end_effector_position[2])])

# Remove before submtion for our refrenc only

## The Three Transformation Matrices

T_base:
- Transforms from the "world" coordinate system to our robot's base coordinate system.
- Accounts for where our robot is positioned/oriented in the world.

T_final:
- The transformation we calculate using D‑H parameters.
- Goes from the robot's base frame to the end‑effector frame.
- This is the forward kinematics computes we did in lab 2 part 4.

T_tool:
- Transforms from the end‑effector frame to the actual tool frame.
- Accounts for any gripper attached to the end‑effector.

Overall Transformation:
- The overall transformation from the world frame to the tool frame is given by:
    $$T_{\text{total}} = T_{\text{base}} \; T_{\text{final}} \; T_{\text{tool}}$$
- Since our world frame is the same as the robot base frame then T_base is the identity matrix. 
- So, if we have a tool attached, the overall transformation simplifies to:
  $$T_{\text{total}} = T_{\text{final}} \; T_{\text{tool}}$$
- If no tool is attached, T_tool is also the identity matrix, and $$T_{\text{total}} = T_{\text{final}}$$


In [ ]:
def position_error(q_position, x_target, y_target, z_target):
    # Calculate forward kinematics for the first three links
    T_values = overall_transformation(dh_table)
    # Extract the end-effector position from the transformation matrix
    X, Y, Z = T_values[?,?], T_values[?,?], T_values[?,?]
    #return an array of differences between calculated and target positions
return[?,?,?]

In [ ]:
def orientation_error(q_orientation, rx_d, ry_d, rz_d):
    # Calculate forward kinematics for the last three links
    T_values = forward_kinematics_func(0, 0, 0, q_orientation[?], q_orientation[?], q_orientation[?])
# Complete the trig equations
#  Extract the end-effector orientation from the transformation matrix
    roll, pitch, yaw = np.arctan2(?,?), np.arctan2(-?,sqrt(?+?)), np.arctan2(?,?)
#return an array of differences between calculated and target orientations
    return [0,0,0]